In [3]:
import pandas as pd
import scipy.io
import numpy as np
import h5py
import pickle
from sklearn.model_selection import train_test_split

X_test = pd.read_pickle('X_test.pkl')
y_test = pd.read_pickle('y_test.pkl')

X_test= np.array(X_test)
X_test= X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [5]:
import matplotlib.pyplot as plt
from keras import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, Dense, BatchNormalization, Input, Lambda, Activation, Conv2D, MaxPooling2D, Reshape, Bidirectional, TimeDistributed, GRU, GlobalMaxPooling1D
from keras.layers import concatenate
from keras.models import Model
from keras.utils import plot_model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, CSVLogger, EarlyStopping
from keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from matplotlib import pyplot


# load model

model= load_model('ALSNet3.hdf5')

# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=1)
#yhat_probs= 1-yhat_probs

# measure best fscore
#y_test=1-y_test
precision, recall, thresholds = precision_recall_curve(y_test, yhat_probs)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
#print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], fscore[ix], precision[ix], recall[ix]))


# predict classes for threshold
yhat_classes = yhat_probs >= 0.5
cm = confusion_matrix(y_test,yhat_classes)
tn, fp, fn, tp = cm.ravel()

model_precision= tp/(tp+fp)
model_recall= tp/(tp+fn)
model_f= 2*model_precision*model_recall/(model_precision+model_recall)
model_accuracy= (tp+tn)/(tp+fp+tn+fn)
model_sensitivity= model_recall
model_specificity= tn/(tn+fp)

# print('Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (0.5, model_f, model_precision, model_recall))
print('Accuracy=%f, Sensitivity=%f, Specificity=%f' % (model_accuracy, model_sensitivity, model_specificity))
print('Confusion matrix : \n',cm)

/Users/prashantronad/Library/Python/3.9/lib/python/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


21/21 ━━━━━━━━━━━━━━━━━━━━ 11s 541ms/step
Accuracy=0.977444, Sensitivity=0.967742, Specificity=0.985915
Confusion matrix : 
 [[350   5]
 [ 10 300]]


In [12]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 23437, 1)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv1D)                  │ (None, 23427, 32)      │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 23427, 32)      │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv1D)                  │ (None, 23411, 64)      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_bn (BatchNormalization)   │ (None, 23411, 64)      │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv1D)                  │ (None, 23393, 128)     │        57,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3_bn (BatchNormalization)   │ (None, 23393, 128)     │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ emg_dense1 (Dense)              │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ emg_dense1_bn                   │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ emg (Dense)                     │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 85,827 (335.27 KB)

 Trainable params: 85,249 (333.00 KB)

 Non-trainable params: 576 (2.25 KB)

 Optimizer params: 2 (12.00 B)

In [6]:
print("Input shape:", X_test.shape)  # (num_samples, num_features, 1)
( X_test[0])  # view one EMG signal sample
for i in range(0, X_test.shape[0]):
    print(X_test[0][i])

print(X_test.shape[0])

Input shape: (665, 23437, 1)
[-20.44708934]
[-18.99748226]
[-18.99748226]
[-16.93751431]
[-17.92935073]
[-17.31898985]
[-18.08194095]
[-17.16639963]
[-19.68413825]
[-17.54787518]
[-15.18272679]
[-15.79308766]
[-16.25085832]
[-18.69230182]
[-16.25085832]
[-13.42793927]
[-15.2590219]
[-9.46059358]
[-9.30800336]
[-14.03830014]
[-13.42793927]
[-13.04646372]
[-12.51239796]
[-9.6894789]
[-8.08728161]
[-5.18806744]
[-8.77393759]
[-7.93469139]
[-5.26436255]
[-8.54505226]
[-5.18806744]
[-5.26436255]
[0.8392462]
[0.76295109]
[2.4414435]
[-1.6021973]
[0.]
[0.53406577]
[1.6021973]
[-0.68665599]
[1.90737774]
[0.61036088]
[-4.27252613]
[1.75478752]
[-0.99183642]
[1.37331197]
[-0.68665599]
[-2.13626307]
[-0.07629511]
[-0.30518044]
[1.98367285]
[0.61036088]
[-1.14442664]
[-0.30518044]
[-0.45777066]
[-2.05996796]
[-2.05996796]
[-3.50957504]
[3.2043946]
[0.61036088]
[0.8392462]
[0.45777066]
[-2.67032883]
[-1.83108263]
[-2.36514839]
[-3.05180438]
[-4.95918212]
[-5.03547723]
[-3.66216526]
[-3.05180438]
[1

In [7]:
print("Output probability (first sample):", yhat_probs[0])
print("Predicted class:", int(yhat_classes[0]))


Output probability (first sample): [2.1067528e-05]
Predicted class: 0


/var/folders/_v/dz3m_2kn1pjgzx82_dbrsy680000gn/T/ipykernel_71783/1809716460.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("Predicted class:", int(yhat_classes[0]))


In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# Load and reshape input
df = pd.read_csv('/Users/prashantronad/Documents/College/6thsem/IDP/output_file_test.csv', header=None)
input_signal = df.iloc[56].to_numpy()

print(len(input_signal))
print(len(df[1]))

# Ensure correct length
if len(input_signal) != 23437:
    raise ValueError(f"Input length is {len(input_signal)}, expected 23437")

input_signal = input_signal.reshape(1, 23437, 1)  # (1 sample, 23437 time steps, 1 channel)

# Load model
model = load_model('ALSNet3.hdf5')

# Predict
yhat_prob = model.predict(input_signal)
yhat_class = (yhat_prob >= 0.5).astype(int)

# Output
print(f"Predicted probability: {yhat_prob[0][0]:.4f}")
print(f"Predicted class: {yhat_class[0][0]}")


/Users/prashantronad/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/prashantronad/Library/Python/3.9/lib/python/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


23437
666
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Predicted probability: 0.7695
Predicted class: 1
